In [1]:
import bs4 as bs
import urllib.request
import bz2
import json
import re

In [2]:
file = bz2.open('RS_2017-01.bz2')
temp = open('allSubmissions.txt','w+')

In [3]:
def correction(e, x, y):
    try :
        e = str(e)[::-1]
        e = int(e[1:e.index(' ')][::-1])
        x = x[:e-3]+'"'+x[e-2:]
        y = json.loads(x, strict = False)
        return y
    except Exception as e :
        y = correction(e, x, y)        

In [ ]:
submissions = []
x = ""
for line in file :
    x = '{'+str(line)[3:-4]+'}'
    x = x.replace('\\\\"','').replace('\\','')
    try :
        y = json.loads(x, strict = False)
    except Exception as e :
        y = correction(e, x, y)
    temp.write(str(y)+'\n')

In [6]:
# x.replace('\\\\"','').replace('\\','')[250:300]
len(submissions)

200001

In [ ]:
for submission in submissions :
    instance_id = 1
    filename = submission['id'] + '.xml'
    f = open(filename,"w+")
    
    f.write("<?xml version='1.0' encoding='utf-8'?>"+"\n")
    f.write("<KnolML>"+"\n")
    f.write('	<KnowledgeData Type="Submision" Id= "'+str(submission['id'])+'">'+"\n")
    
    f.write('		<Title>'+submission['title']+'</Title>'+"\n")
    f.write('		<Instance Id= "'+str(instance_id)+'" InstanceType= "Description">'+"\n")
    
    f.write("			<TimeStamp>"+"\n")
    f.write(" 				<CreationUTC>"+str(submission['created_utc'])+"</CreationUTC> "+"\n")
    f.write("			</TimeStamp>"+"\n")
    
    f.write("			<Contributors>"+"\n")
    f.write(" 				<OwnerUserId>"+submission['author']+"</OwnerUserId> "+"\n")
    f.write("			</Contributors>"+"\n")
    
    f.write("			<Body>"+"\n")
    f.write(" 				<Text>"+"\n")
    f.write\\\\("					"+submission['selftext']+"\n")
    f.write("				</Text> "+"\n")
    f.write("			</Body>"+"\n")
    
    f.write("			<Credit>"+"\n")
    f.write(" 			.replace('\\','')	<Sx[795]core>"+str(submission['score'])+"</Score> "+"\n")
    f.write("			</Credit>"+"\n")
    
    f.write('		</Instance>'+"\n")

    submission_id = submission['id']
    url = 'https://api.pushshift.io/redd1560:1570it/submission/comment_ids/' + submission_id
    sauce = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(sauce, 'lxml')
    
    comment_ids = json.loads(soup.p.string)['data']
    
    for comment_id in comment_ids :
        url = 'http://api.pushshift.io/reddit/search/comment/?ids=' + comment_id
        sauce = urllib.request.urlopen(url).read()
        soup = bs.BeautifulSoup(sauce, 'lxml')
    
        comment_description = json.loads(soup.p.string)['data']
        
        for comment in comment_description :
            instance_id += 1
            f.write('		<Instance Id= "'+str(instance_id)+'" InstanceType= "Comment">'+"\n")
            f.write("			<TimeStamp>"+"\n")
            f.write(" 				<CreationUTC>"+str(comment["created_utc"])+"</CreationUTC> "+"\n")
            f.write("			</TimeStamp>"+"\n")

            f.write("			<Contributors>"+"\n")
            f.write(" 				<OwnerUserId>"+comment["author"]+"</OwnerUserId>"+"\n")
            f.write("			</Contributors>"+"\n")

            f.write("			<Body>"+"\n")
            f.write(" 				<Text>"+"\n")
            f.write("					"+comment["body"]+"\n")
            f.write("				</Text> "+"\n")
            f.write("			</Body>"+"\n")

            f.write("			<Credit>"+"\n")
            f.write(" 				<Score>"+str(comment["score"] if "score" in comment else 0)+"</Score> "+"\n")
            f.write("			</Credit>"+"\n")

            f.write('		</Instance>'+"\n")

    
    f.write('	</KnowledgeData>'+"\n")
    f.write('</KnolML>')